In [1]:
from mimic.utilities.utilities import set_all_seeds
from mimic.utilities.utilities import plot_gMLV
from mimic.model_infer import *
from mimic.model_simulate import *
import pandas as pd


import numpy as np


import matplotlib.pyplot as plt


from scipy.integrate import odeint


from numpy import linalg as la

import sklearn.linear_model


from sklearn.linear_model import LinearRegression


from sklearn.linear_model import Ridge


from sklearn.linear_model import Lasso


from sklearn.linear_model import ElasticNet, ElasticNetCV


from sklearn.model_selection import RepeatedKFold


from sklearn.model_selection import cross_val_score


from sklearn.model_selection import KFold


from sklearn.model_selection import GridSearchCV


import sys


sys.path.append("../../")

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Simulate some time course data from the gMLV

### Five species, six metabolites, single time course

This model assumes metabolite production is associated with abundance:  dS/dt = alpha X <br>
Note that this model needs rethinking as it cannot handle negative productivities <br>
In this simple example we don't need to infer the time course. We just linearize and estimate the elements of alpha with Lasso<br>
Number of metabolites is 6 here

In [2]:
# Simulate some microbiota and metabolites
set_all_seeds(1234)

# SETUP MODEL
# establish size of model
num_species = 5
num_metabolites = 6

# construct interaction matrix
# TODO do this programmatically
M = np.zeros((num_species, num_species))
np.fill_diagonal(M, [-0.05, -0.1, -0.15, -0.01, -0.2])
# M[0,2] = -0.025
M[1, 3] = 0.05
# M[4,0] = 0.02

# construct growth rates matrix
mu = np.random.lognormal(0.01, 0.5, num_species)

# construct metabolite production matrix
alpha = np.zeros((num_metabolites, num_species))
alpha[1, 4] = 1
alpha[4, 2] = -0.5

# instantiate simulator
simulator = sim_gMLV(num_species=num_species,
                     num_metabolites=num_metabolites,
                     M=M,
                     mu=mu,
                     beta=alpha)
simulator.print()

# PRODUCE SIMULATED RESULTS
# initial conditions
init_species = 10 * np.ones(num_species)
init_metabolites = 10 * np.ones(num_metabolites)

times = np.arange(0, 5, 0.1)
yobs, sobs, sy0, _, _, _ = simulator.simulate(
    times=times, sy0=np.hstack((init_species, init_metabolites)))

# add some gaussian noise
yobs = yobs + np.random.normal(loc=0, scale=0.1, size=yobs.shape)
sobs = sobs + np.random.normal(loc=0, scale=0.1, size=sobs.shape)

# plot simulation
plot_gMLV(yobs, sobs, times)

TypeError: Can't instantiate abstract class sim_gMLV with abstract method set_parameters